<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con Spacy utilizando un corpus de wikipedia


In [1]:
import bs4 as bs  # Para leer y parsear el texto en HTML de wikipedia
import gradio as gr
import json
import lxml
import nltk
import numpy as np
import random
import re
import spacy
import ssl
import string
import urllib.request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.es.stop_words import STOP_WORDS as es_stop

In [2]:
context = ssl._create_unverified_context()

In [3]:
nlp = spacy.load('es_core_news_sm')


### Datos
Se consumirán los datos del artículo de wikipedia sobre las 3 leyes de la robótica de Isaac Asimov.

In [76]:
url = 'https://psicologiaymente.com/cultura/leyes-de-robotica'
req = urllib.request.Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
raw_html = urllib.request.urlopen(req, context=context).read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

In [77]:
# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [78]:
# Saco primer parte de la página
article_text = article_text[118:]

In [79]:
article_text = article_text[:-1158]

In [80]:
article_text

'¿conoces las leyes de la robótica? son tres leyes que desarrolló isaac asimov, el célebre escritor de ficción ruso, autor de obras tan conocidas como “yo, robot”.en este artículo veremos en qué consisten sus tres leyes de la robótica, cómo nacieron, qué características tienen, cómo podemos interpretarlas y cuál ha sido la repercusión de isaac asimov.isaac asimov nació el 2 de enero de 1920 en petróvichi, rusia, y falleció el 6 de abril de 1992 en nueva york (eeuu), a la edad de 72 años. asimov fue un escritor de ciencia ficción de origen ruso y nacionalizado estadounidense, además de profesor de bioquímica de la facultad de medicina de la universidad de boston.se hizo mundialmente conocido por ser autor de numerosas historias de ciencia ficción, pero también por su obra de historia y divulgación científica.las tres leyes de la robótica fueron descritas por isaac asimov, el famoso escritor de ciencia ficción. dichas leyes se aplicaban a los robots que aparecen en sus novelas y cuentos 

In [81]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 5835


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [82]:
# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres: 
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes

text = re.sub(r'\.', r'. ', text)  # Habia algunos puntos sin estar seguidos por espacio

text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

text = re.sub(r'\u200b', '', text) # elimino zero width space

text = re.sub(r'  ', ' ', text)  # Borro espacios dobles

# Tildes y Diacríticas
text = re.sub('á', 'a', text)
text = re.sub('é', 'e', text)
text = re.sub('í', 'i', text)
text = re.sub('ó', 'o', text)
text = re.sub('ú', 'u', text)
text = re.sub('ü', 'u', text)
text = re.sub('ñ', 'n', text)


In [83]:
# Demos un vistazo
text

'¿conoces las leyes de la robotica? son tres leyes que desarrollo isaac asimov, el celebre escritor de ficcion ruso, autor de obras tan conocidas como “yo, robot”. en este articulo veremos en que consisten sus tres leyes de la robotica, como nacieron, que caracteristicas tienen, como podemos interpretarlas y cual ha sido la repercusion de isaac asimov. isaac asimov nacio el 2 de enero de 1920 en petrovichi, rusia, y fallecio el 6 de abril de 1992 en nueva york (eeuu), a la edad de 72 anos. asimov fue un escritor de ciencia ficcion de origen ruso y nacionalizado estadounidense, ademas de profesor de bioquimica de la facultad de medicina de la universidad de boston. se hizo mundialmente conocido por ser autor de numerosas historias de ciencia ficcion, pero tambien por su obra de historia y divulgacion cientifica. las tres leyes de la robotica fueron descritas por isaac asimov, el famoso escritor de ciencia ficcion. dichas leyes se aplicaban a los robots que aparecen en sus novelas y cuen

In [84]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 5860


### 3 - Tokenización

In [85]:
doc = nlp(text)
lexical_tokens = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
lexical_tokens

['conoces',
 'leyes',
 'robotica',
 'leyes',
 'desarrollo',
 'isaac',
 'asimov',
 'celebre',
 'escritor',
 'ficcion',
 'ruso',
 'autor',
 'obras',
 'conocidas',
 'robot',
 'articulo',
 'veremos',
 'consisten',
 'leyes',
 'robotica',
 'nacieron',
 'caracteristicas',
 'interpretarlas',
 'repercusion',
 'isaac',
 'asimov',
 'isaac',
 'asimov',
 'nacio',
 '2',
 'enero',
 '1920',
 'petrovichi',
 'rusia',
 'fallecio',
 '6',
 'abril',
 '1992',
 'york',
 'eeuu',
 'edad',
 '72',
 'anos',
 'asimov',
 'escritor',
 'ciencia',
 'ficcion',
 'origen',
 'ruso',
 'nacionalizado',
 'estadounidense',
 'profesor',
 'bioquimica',
 'facultad',
 'medicina',
 'universidad',
 'boston',
 'mundialmente',
 'conocido',
 'autor',
 'numerosas',
 'historias',
 'ciencia',
 'ficcion',
 'obra',
 'historia',
 'divulgacion',
 'cientifica',
 'leyes',
 'robotica',
 'descritas',
 'isaac',
 'asimov',
 'famoso',
 'escritor',
 'ciencia',
 'ficcion',
 'dichas',
 'leyes',
 'aplicaban',
 'robots',
 'aparecen',
 'novelas',
 'cuento

In [86]:
corpus = []
for sent_id, sent in enumerate(doc.sents):
    corpus.append(str(sent))

### 4 - Lematización, Remoción de Simbolos de puntuación
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [87]:
# Tokenization & lemmatization
lemma_list = []
for token in doc:
    lemma_list.append(token.lemma_)
print(lemma_list)

['¿', 'conocer', 'el', 'ley', 'de', 'el', 'robotica', '?', 'ser', 'tres', 'ley', 'que', 'desarrollo', 'isaac', 'asimov', ',', 'el', 'celebre', 'escritor', 'de', 'ficcion', 'ruso', ',', 'autor', 'de', 'obra', 'tanto', 'conocido', 'como', '“', 'yo', ',', 'robot', '”', '.', 'en', 'este', 'articulo', 'ver', 'en', 'que', 'consistar', 'su', 'tres', 'ley', 'de', 'el', 'robotica', ',', 'como', 'nacer', ',', 'que', 'caracteristica', 'tener', ',', 'como', 'poder', 'interpretar él', 'y', 'cual', 'haber', 'ser', 'el', 'repercusion', 'de', 'isaac', 'asimov', '.', 'isaac', 'asimov', 'nacio', 'el', '2', 'de', 'enero', 'de', '1920', 'en', 'petrovichi', ',', 'rusia', ',', 'y', 'fallecer', 'el', '6', 'de', 'abril', 'de', '1992', 'en', 'nueva', 'york', '(', 'eeuu', ')', ',', 'a', 'el', 'edad', 'de', '72', 'ano', '.', 'asimov', 'ser', 'uno', 'escritor', 'de', 'ciencia', 'ficcion', 'de', 'origen', 'ruso', 'y', 'nacionalizado', 'estadounidense', ',', 'adema', 'de', 'profesor', 'de', 'bioquimica', 'de', 'el'

In [88]:
# Stop words
filtered_sentence =[]
for word in lemma_list:
    # word es un string, para recuperar la información de los objetos de SpaCy
    # necesitamos usar el string para pasar a un lexema, el objeto de SpaCy
    # que para cada término contiene la información del preprocesamiento
    # (se podría también directamente filtrar stopwords en el paso de lematización)
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        filtered_sentence.append(word)
filtered_sentence 

['¿',
 'ley',
 'robotica',
 '?',
 'ley',
 'desarrollo',
 'isaac',
 'asimov',
 ',',
 'celebre',
 'escritor',
 'ficcion',
 'ruso',
 ',',
 'autor',
 'obra',
 'conocido',
 '“',
 ',',
 'robot',
 '”',
 '.',
 'articulo',
 'consistar',
 'ley',
 'robotica',
 ',',
 'nacer',
 ',',
 'caracteristica',
 ',',
 'interpretar él',
 'repercusion',
 'isaac',
 'asimov',
 '.',
 'isaac',
 'asimov',
 'nacio',
 '2',
 'enero',
 '1920',
 'petrovichi',
 ',',
 'rusia',
 ',',
 'fallecer',
 '6',
 'abril',
 '1992',
 'york',
 '(',
 'eeuu',
 ')',
 ',',
 'edad',
 '72',
 'ano',
 '.',
 'asimov',
 'escritor',
 'ciencia',
 'ficcion',
 'origen',
 'ruso',
 'nacionalizado',
 'estadounidense',
 ',',
 'adema',
 'profesor',
 'bioquimica',
 'facultad',
 'medicina',
 'universidad',
 'boston',
 '.',
 'mundialmente',
 'conocido',
 'autor',
 'numeroso',
 'historia',
 'ciencia',
 'ficcion',
 ',',
 'obra',
 'historia',
 'divulgacion',
 'cientifico',
 '.',
 'ley',
 'robotica',
 'describir',
 'isaac',
 'asimov',
 ',',
 'famoso',
 'escrito

In [53]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [89]:
# Filter punctuation
filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation+'—¿¡“”']
filtered_sentence

['ley',
 'robotica',
 'ley',
 'desarrollo',
 'isaac',
 'asimov',
 'celebre',
 'escritor',
 'ficcion',
 'ruso',
 'autor',
 'obra',
 'conocido',
 'robot',
 'articulo',
 'consistar',
 'ley',
 'robotica',
 'nacer',
 'caracteristica',
 'interpretar él',
 'repercusion',
 'isaac',
 'asimov',
 'isaac',
 'asimov',
 'nacio',
 '2',
 'enero',
 '1920',
 'petrovichi',
 'rusia',
 'fallecer',
 '6',
 'abril',
 '1992',
 'york',
 'eeuu',
 'edad',
 '72',
 'ano',
 'asimov',
 'escritor',
 'ciencia',
 'ficcion',
 'origen',
 'ruso',
 'nacionalizado',
 'estadounidense',
 'adema',
 'profesor',
 'bioquimica',
 'facultad',
 'medicina',
 'universidad',
 'boston',
 'mundialmente',
 'conocido',
 'autor',
 'numeroso',
 'historia',
 'ciencia',
 'ficcion',
 'obra',
 'historia',
 'divulgacion',
 'cientifico',
 'ley',
 'robotica',
 'describir',
 'isaac',
 'asimov',
 'famoso',
 'escritor',
 'ciencia',
 'ficcion',
 'ley',
 'aplicar',
 'robot',
 'aparecer',
 'novela',
 'cuento',
 'ciencia',
 'ficcion',
 'robot',
 'teniar',


### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [90]:
def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(stop_words=list(es_stop))

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada.

In [92]:
def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text"
)

iface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Q: asimov
A: despues de las tres leyes de la robotica, llego la ley cero, tambien propuesta por asimov.
Q: isaac asimov
A: las tres leyes de la robotica fueron descritas por isaac asimov, el famoso escritor de ciencia ficcion.
Q: rusia
A: isaac asimov nacio el 2 de enero de 1920 en petrovichi, rusia, y fallecio el 6 de abril de 1992 en nueva york (eeuu), a la edad de 72 anos.
Q: ciencia ficcion
A: las tres leyes de la robotica fueron descritas por isaac asimov, el famoso escritor de ciencia ficcion.
Q: dano
A: de esta manera, la primera ley de la robotica hace alusion no solo al hecho de no causar dano, sino tambien al de evitarlo.
Q: danar
A: vamos a conocerlas:la primera ley de la robotica de asimov dice asi: “un robot no puede danar a un ser humano ni, por inaccion, permitir que un ser humano sufra dano”.
Q: robot
A: finalmente, la tercera ley de la robotica dice: “un robot debe proteger su propia existencia en la medida en que ello no entre en conflicto con la primera o la segunda 

(<gradio.routes.App at 0x12fcbe800>, 'http://127.0.0.1:7860/', None)

### 7 -  Conclusiones

Se implementó un BOT que busca la parte del artículo que más se relaciona con nuestro input, utilizando el criterio de similitud de coseno del vector TF-IDF.  
  
Para la implementación se utilizaron técnicas de preprocesamiento:
1. Regular Expresions para reemplazo de caracteres especiales, espacios dobles, etc.
2. Tokenización
3. Extracción de oraciones
4. Lematización
5. Filtrado de stop words y signos de puntuación  
  


##### Acerca de los resultados

Es interesante ver algunos resultados:
- Cuando el input fue Asimov o Isaac Asimov, la respuesta tuvo que ver con las leyes propuestas por él. Cuando el input fue Rusia, la respuesta tuvo que ver con el origen del escritor.
- Para Isaac Asimov o Ciencia Ficción, la respuesta es la misma.
- Si se pasa primer o primera, el sistema no entiende de que se está hablando. Pero si se pasa inacción, daño o dañar, el sistema entiende que se habla de la 1er ley.
- Si se pasa segunda, el sistema no entiende el input. Pero si se pasa órdenes, el sistema interpreta que se habla de la 2da ley.
- Cuando se le pasa robot, el sistema lo relaciona con la 3er ley.
- Si se pasa dañar humanidad, el sistema interpreta que se habla de la 4ta ley (o ley 0).
  
Recordando las 4 leyes:
<ol start="0">
  <li>Un robot no puede dañar a la humanidad ni permitir que sufra algún daño por inacción.</li>
  <li>Un robot no puede dañar a un ser humano ni, por inacción, permitir que un ser humano sufra daño.</li>
  <li>Un robot debe cumplir las órdenes de los seres humanos, excepto si dichas órdenes entran en conflicto con la Primera Ley.</li>
  <li>Un robot debe proteger su propia existencia en la medida en que ello no entre en conflicto con la Primera o la Segunda Ley.</li>
</ol>





En base a estos resultados, se puede comprobar que la técnica es sencilla pero efectiva.